# Deep Research Agent: Core Workflow

This notebook demonstrates the core research workflow of the Deep Research Agent, including:
1. Query Decomposition
2. Web Search & RAG Ingestion
3. Evidence Auditing & Verification
4. Final Synthesis
5. Artifact Export

In [ ]:
import sys
import os
import subprocess

def setup_environment():
    """Setup environment for Google Colab or local execution."""
    try:
        import google.colab
        IN_COLAB = True
    except ImportError:
        IN_COLAB = False

    if IN_COLAB:
        print("Detected Google Colab environment. Installing dependencies...")
        
        # 1. Uninstall conflicting packages (protobuf versions)
        subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "google-ai-generativelanguage", "tensorflow", "grpcio-status"])
        
        # 2. Clone repository if needed
        if not os.path.exists("gemini-fullstack-langgraph-quickstart"):
            subprocess.check_call(["git", "clone", "https://github.com/GoogleCloudPlatform/gemini-fullstack-langgraph-quickstart.git"])
            os.chdir("gemini-fullstack-langgraph-quickstart")
        elif os.path.exists("backend"): 
            # Already in root
            pass
        
        # 3. Install backend with strict dependency resolution
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-e", "backend"])
        
        print("Dependencies installed.")
    else:
        print("Running locally. Assuming dependencies are installed.")

setup_environment()

# Add backend/src to path regardless of install method to ensure modules are found
# This fixes the ModuleNotFoundError even if pip install -e . is used but kernel not restarted or paths not propagated
if os.path.exists("backend/src"):
    sys.path.append(os.path.abspath("backend/src"))
elif os.path.exists("../backend/src"):
    sys.path.append(os.path.abspath("../backend/src"))
elif os.path.exists("gemini-fullstack-langgraph-quickstart/backend/src"):
    sys.path.append(os.path.abspath("gemini-fullstack-langgraph-quickstart/backend/src"))

print(f"Current sys.path: {sys.path}")

## 1. Setup & Dependencies

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

# Add backend/src to path so we can import modules
sys.path.append(os.path.abspath("../backend/src"))

# Set API Keys (User should replace these or set in env)
# os.environ["GOOGLE_API_KEY"] = "..."
# os.environ["TAVILY_API_KEY"] = "..."

In [ ]:
from agent.deep_search_agent import DeepSearchAgent
from langchain_google_genai import ChatGoogleGenerativeAI
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## 2. Initialize Agent Components

In [ ]:
# Initialize LLM
# Using Gemini as per repo config, assuming env vars are set
try:
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)
except Exception as e:
    print(f"Error initializing LLM: {e}")
    # Fallback for testing without API keys (Mock)
    class MockLLM:
        def invoke(self, prompt): return "Mock response"
        def generate(self, prompt): return "Mock response"
    llm = MockLLM()

# Initialize Agent
agent = DeepSearchAgent(llm_client=llm)
print("Agent initialized.")

## 3. Single Query Research Demo

In [ ]:
query = "Latest advances in renewable energy storage"
print(f"Starting research on: {query}")

final_answer = agent.research(query)

print("\n=== FINAL ANSWER ===\n")
print(final_answer)

## 4. RAG Verification Deep Dive

Inspect the internal state of the RAG system to verify ingestion and retrieval.

In [ ]:
# Inspect Doc Store
print(f"Total documents indexed: {len(agent.rag.doc_store)}")

# Sample a document
if len(agent.rag.doc_store) > 0:
    doc_id = list(agent.rag.doc_store.keys())[0]
    print(f"\nSample Document ({doc_id}):")
    print(agent.rag.doc_store[doc_id])

In [ ]:
# Test Retrieval
test_query = "battery cost reduction"
results = agent.rag.retrieve(test_query, top_k=3)

print(f"\nRetrieval results for '{test_query}':")
for doc, score in results:
    print(f"- [{score:.2f}] {doc.content[:100]}...")

## 5. Export Research Artifacts

In [ ]:
state = agent.rag.export_state()
print("RAG State Export:", state)

# Retrieve all docs for export
all_docs = agent.get_retrieved_documents()
print(f"\nPrepared {len(all_docs)} documents for export.")